In [4]:
import requests
import os
from bs4 import BeautifulSoup

def download_one_episode(title, no, url):
    """
    하나의 웹툰 에피소드 이미지를 다운로드하여 폴더에 저장하는 함수

    Args:
        title (str): 웹툰의 제목. 폴더명으로 사용됩니다.
        no (str or int): 회차 번호. 하위 폴더명으로 사용됩니다.
        url (str): 다운로드할 회차의 상세 페이지 URL.
    """
    print(f"'{title}' - {no}화 다운로드를 시작합니다.")

    # --- 1. 폴더 생성 ---
    # 웹툰 제목으로 메인 폴더를, 회차 번호로 하위 폴더를 생성합니다.
    # exist_ok=True 옵션은 폴더가 이미 있어도 에러를 발생시키지 않습니다.
    dir_path = os.path.join(title, f"{no:04d}") # 4자리 숫자로 포맷 (예: 48 -> 0048)
    os.makedirs(dir_path, exist_ok=True)
    
    # --- 2. 웹툰 페이지 HTML 가져오기 ---
    # 네이버 웹툰은 이미지 요청 시 referer(어느 페이지에서 접속했는지)를 확인합니다.
    req_header = {
        'referer': url, # 현재 접속한 페이지 주소를 referer로 설정
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"
    }
    
    res = requests.get(url, headers=req_header)
    if not res.ok:
        print(f"페이지를 불러오는 데 실패했습니다. (상태 코드: {res.status_code})")
        return

    # --- 3. HTML에서 이미지 URL 리스트 추출 ---
    soup = BeautifulSoup(res.content, "html.parser")
    
    # 웹툰 이미지는 보통 'div.wt_viewer' 안에 있습니다.
    img_tags = soup.select("div.wt_viewer img")
    
    if not img_tags:
        print("이미지를 찾을 수 없습니다. 웹툰 페이지의 구조가 변경되었을 수 있습니다.")
        return
        
    img_urls = [tag['src'] for tag in img_tags]
    print(f"총 {len(img_urls)}개의 이미지를 발견했습니다.")

    # --- 4. 각 이미지를 순서대로 다운로드 ---
    for i, img_url in enumerate(img_urls):
        print(f"{i+1}/{len(img_urls)} 다운로드 중: {os.path.basename(img_url)}")
        
        # 이미지 URL로 get 요청 보내기 (헤더 포함)
        img_res = requests.get(img_url, headers=req_header)
        
        if img_res.ok:
            # 이미지 데이터를 바이너리(binary) 형태로 가져오기
            img_data = img_res.content
            
            # 파일명 설정 (예: 001.jpg, 002.jpg, ...)
            # os.path.splitext로 확장자를 분리하고 순번을 붙여줍니다.
            _, ext = os.path.splitext(img_url)
            file_name = f"{i+1:03d}{ext}" # 3자리 숫자로 포맷 (예: 1 -> 001)
            file_path = os.path.join(dir_path, file_name)
            
            # 바이너리 쓰기 모드('wb')로 파일을 열고 이미지 데이터 저장
            with open(file_path, 'wb') as file:
                file.write(img_data)
        else:
            print(f"이미지 다운로드 실패 (상태 코드: {img_res.status_code})")
            
    print(f"\n다운로드 완료! -> '{dir_path}' 폴더를 확인하세요.")



In [3]:
webtoon_title = '전지적 독자 시점'
episode_no = 210
episode_url = f'https://comic.naver.com/webtoon/detail?titleId=747269&no={episode_no}'

download_one_episode(webtoon_title, episode_no, episode_url)


'전지적 독자 시점' - 210화 다운로드를 시작합니다.
총 67개의 이미지를 발견했습니다.
1/67 다운로드 중: 20240507150807_92a1c7f1d629a56d5c716ee60871f8e2_IMAG01_1.jpg
2/67 다운로드 중: 20240507150807_92a1c7f1d629a56d5c716ee60871f8e2_IMAG01_2.jpg
3/67 다운로드 중: 20240507150807_92a1c7f1d629a56d5c716ee60871f8e2_IMAG01_3.jpg
4/67 다운로드 중: 20240507150807_92a1c7f1d629a56d5c716ee60871f8e2_IMAG01_4.jpg
5/67 다운로드 중: 20240507150807_92a1c7f1d629a56d5c716ee60871f8e2_IMAG01_5.jpg
6/67 다운로드 중: 20240507150807_92a1c7f1d629a56d5c716ee60871f8e2_IMAG01_6.jpg
7/67 다운로드 중: 20240507150807_92a1c7f1d629a56d5c716ee60871f8e2_IMAG01_7.jpg
8/67 다운로드 중: 20240507150807_92a1c7f1d629a56d5c716ee60871f8e2_IMAG01_8.jpg
9/67 다운로드 중: 20240507150807_92a1c7f1d629a56d5c716ee60871f8e2_IMAG01_9.jpg
10/67 다운로드 중: 20240507150807_92a1c7f1d629a56d5c716ee60871f8e2_IMAG01_10.jpg
11/67 다운로드 중: 20240507150807_92a1c7f1d629a56d5c716ee60871f8e2_IMAG01_11.jpg
12/67 다운로드 중: 20240507150807_92a1c7f1d629a56d5c716ee60871f8e2_IMAG01_12.jpg
13/67 다운로드 중: 20240507150807_92a1c7f1d629a56d5c716ee60